In [2]:
import os
import pandas as pd
import numpy as np
from radiomics import featureextractor

In [3]:
data_path = "../../data/"

In [8]:
params = data_path + 'Params.yaml'

extractor = featureextractor.RadiomicsFeatureExtractor(params)

features = {}
diagnostic = []

target_class = pd.read_csv(data_path + 'target_class.csv')
target_class_dict = dict(zip(target_class['CLID'], target_class['Pam50.Call']))

base_path = data_path + r'TCGA_Segmented_Lesions_UofC/'
les_files = sorted(os.listdir(base_path))

les_files = [f for f in les_files if f.endswith('.les')]

for t, les_file in enumerate(les_files):
    lesion_path = os.path.join(base_path, les_file)
    print(f"Processing: {lesion_path}")

    lesion_name = os.path.splitext(les_file)[0]
    lesion_name_base = "-".join(lesion_name.split("-")[:3])

    if lesion_name_base in target_class_dict:
        diagnostic.append(target_class_dict[lesion_name_base])
    else:
        print(f"Warning: Lesion {lesion_name_base} not found in target_class.csv")
        continue

    mask = sitk.ReadImage(lesion_path)

    features[t] = extractor.execute(mask, mask, label=1)

feature_names = list(sorted(filter(lambda k: k.startswith("original_"), features[0])))

# Make a numpy array of all the values
samples = np.zeros((len(features), len(feature_names)))
for case_id, case_features in features.items():
    a = np.array([case_features[feature_name] for feature_name in feature_names])
    samples[case_id, :] = a

# Replace NaNs with zeros
samples = np.nan_to_num(samples)

print("Feature extraction complete.")

Processing: ../../data/TCGA_Segmented_Lesions_UofC/TCGA-AO-A03M-1.les
['LumA']


RuntimeError: Exception thrown in SimpleITK ImageFileReader_Execute: /home/conda/feedstock_root/build_artifacts/libsimpleitk_1627514680097/work/Code/IO/src/sitkImageReaderBase.cxx:105:
sitk::ERROR: Unable to determine ImageIO reader for "../../data/TCGA_Segmented_Lesions_UofC/TCGA-AO-A03M-1.les"